# 1. Standard Streams

Every process — every running instance of a command - interacts with its environment by means of communication channels called streams. In *nix systems, processes commonly start with three special kinds of streams called standard streams:

* standard input (which is used to receive input)
* standard output (where command output goes)
* standard error (where error messages go)

They are often abbreviated as stdin, stdout, and stderr, respectively.

Let's recall the last example in the previous lesson:

    /home/learn$ rm /dev/null
    rm: cannot remove ‘/dev/null’: Permission denied
    
A process was instantiated by running rm /dev/null. If we run rm /dev/null one time, that's one process. If we run it a second time, that's another process. If two different users run it simultaneously, that's two different processes.

Nowadays, things are a little bit different — they're all connected to the shell, and the shell will take care of emulating this behavior for us. You may recall from the first lesson in the first course that one of the names for the terminal is "terminal emulator", this is why.


![](https://dq-content.s3.amazonaws.com/392/Standard+Streams+2.png)

Up until the previous screen, we handled stdin and stdout only. We began by redirecting stdout to files, and then learned to pipe stdout of one command into the standard input of another command.

The message we got above — rm: cannot remove ‘/dev/null’: Permission denied — is an example of a standard error, which is a different stream than stdout. That's why our try at redirecting the output of rm /dev/null in the last lesson didn't work: we redirected stdout and not stderr.

Redirecting standard errors requires its own syntax. Let's continue with our example. To redirect stderr to a file, we need to precede the > symbol with the number 2, like this:

    /home/learn$ rm /dev/null 2>err
    /home/learn$ cat err

# 2. Recap

Now that we know about standard streams, let's recap what we know about redirection in this new light.

We learned how to redirect standard output with command >filename. Running command spawns a process with the usual standard streams, while >filename redirects the output from the shell to the file filename. Diagramatically it is represented as follows.

![](https://dq-content.s3.amazonaws.com/392/Recap+1.png)

Similarly to the above, 2>filename makes it so that the standard error of command is redirected to filename:

![](https://dq-content.s3.amazonaws.com/392/Recap+2.png)


We also learned how to use the output of a command, say, cmd1, as the input of another command, say, cmd2 by running cmd1 | cmd2. The following diagram represents this procedure.


![](https://dq-content.s3.amazonaws.com/392/Recap+3.png)

# 3. Redirecting Two Streams

We've seen how to redirect stdout and stderr, but only one at a time. To redirect both, we just need to include the instruction for both. Here's an example.

    /home/learn$ ls east south

In [2]:
%%bash

ls /dev/null /home/inexistent >all_output 2>all_output

cat all_output

/dev/null
 access '/home/inexistent': No such file or directory


# 4. File Descriptors

We mentioned earlier that processes commonly start with the standard streams that we learned about. More specifically, these are inherited from the parent process.

A parent process is a process that spawned another process. For instance, we are using the Bash shell in this course, when we run a command, say, date, this spawns a new process that is a child process of the Bash process. In this situation, we say that Bash is a parent process of the process spawned by date.

The names stdin, stdout and stderr exist for human convenience. The shell does not know them by that name. Rather, for each process, the operating system will create a table that will refer to these streams by non-negative integers called file descriptors (fd).

Typically, we have the following correspondence.

| File Descriptor | Stream |
|-----------------|--------|
| 0               | stdin  |
| 1               | stdout |
| 2               | stderr |

And now we see why the syntax to redirect stderr is 2> — it refers to the process file descriptor 2.

We also learned that to redirect stdout, we should use >. This is, in fact, a bit of syntactic sugar. The full notation is 1>, which according to the table above and to what we learned, refers to stdout. So, for example, in our previous example, when we ran ls east south >compass 2>compass, we could have ran ls east south 1>compass 2>compass instead.

Before we move on, let's see two cool things. We'll run ls -l /proc/\$$/fd to list the files of the directory /proc/\$$/fd in long format. Here is the output:


    total 0                                                                         
    lrwx------ 1 dq dq 64 May 19 19:19 0 -> /dev/pts/0
    lrwx------ 1 dq dq 64 May 19 19:19 1 -> /dev/pts/0
    lrwx------ 1 dq dq 64 May 19 19:19 2 -> /dev/pts/0
    lrwx------ 1 dq dq 64 May 19 19:19 255 -> /dev/pts/0
    
The /proc/\$$ part of the path refers to the directory of the current process. In this case, it is our own Bash shell. And the fd directory is the file descriptor directory. So we see that file descriptors are also represented as files (remember that in Unix-like systems everything is a file).

These files are symbolic links (think of them as shortcuts) to the file /dev/pts/0. And again, in classic "everything is a file" fashion, /dev/pts/0 is the file that represents our shell!

Remember how we learned that the standard streams are connected to the shell? What we just saw verifies this: the file descriptor files are linked to the file that represents our shell.

Processes can have many file descriptors (we can see fd 255 above) and we'll discuss the ones corresponding to the standard streams.

There's no exercise in this screen. Feel free to experiment and move on when you're ready.



# 5. A New Point of View

We can now look at redirection with new eyes. The picture painted in this screen more closely resembles what really happens under the hood.

Our shell's file descriptor table begins in the state represented below.

![](https://dq-content.s3.amazonaws.com/392/A+New+Point+of+View+1.png)

When we run command 1>filename (which is the same as command >filename), the resulting process will inherit the file descriptor table from its parent process, i.e. our shell, and 1>filename will alter the file descriptor table to the following state.

![](https://dq-content.s3.amazonaws.com/392/A+New+Point+of+View+2.png)

Similarly, when we run command 2>filename, the file descriptor table transforms from the initial state into this:

![](https://dq-content.s3.amazonaws.com/392/A+New+Point+of+View+3.png)

And piping command1 | command2 is represented by the following diagram, where the process spawned by command1 is on the left, and the one spawned by command2 is on the right.

![](https://dq-content.s3.amazonaws.com/392/A+New+Point+of+View+4.png)

# 6. Duplicating File Descriptors

A few screens ago we mentioned that the shell provides us with a way to redirect both stdout and stderr to a file directly. We can do this by duplicating file descriptors. Let's look into what duplicating file descriptors means.

Running command 2>&1 will copy file descriptor 1 into 2. This means that whatever is written to fd 2 will go to where fd 1 goes. The file descriptor table would then be represented by the following diagram.

![](https://dq-content.s3.amazonaws.com/392/Duplicating+File+Descriptors+1.png)

Since the shell makes it so that both fd 1 and fd 2 print to screen, this would make no difference. In fact, the diagram above is equivalent to the diagram we used to represent the initial state:

![](https://dq-content.s3.amazonaws.com/392/Duplicating+File+Descriptors+2.png)

A more interesting example is using file descriptor duplication to redirect both stdout and stderr to a file. We can achieve this by running command >filename 2>&1. Let's break this down.

Redirecting 1 into a file (>filename), makes the file descriptor table look like this (remember we always start from the initial state):

![](https://dq-content.s3.amazonaws.com/392/Duplicating+File+Descriptors+3.png)

We then create a copy of 1 in 2 (2>&1), so that we get the following file descriptor table diagram:



![](https://dq-content.s3.amazonaws.com/392/Duplicating+File+Descriptors+4.png)


In [5]:
%%bash

ls /dev/null /home/inexistant >all_output_v2 2>&1

echo "Test" > /dev/null 2>&1

# 7. Order of Redirections

In [6]:
%%bash

ls /dev/null /home/inexistent 2>&1 1>redirection_order

ls /dev/null /home/inexistent >order_verification

diff -y redirection_order order_verification

ls: cannot access '/home/inexistent': No such file or directory
/dev/null							/dev/null


ls: cannot access '/home/inexistent': No such file or directory


# 8. Standard Input

We've discussed standard output and standard error extensively, and now it's time to discuss input more in-depth.

Most of the commands we've seen so far have been taking input not from standard input, but from the contents of filenames we pass as arguments to the command. This is what happens when we run, for instance, sort east west.

However, most of the commands we learned also take input from standard input. This is characterized by inserting data manually into the shell, instead of passing filenames as arguments. Let's see an example.

Below we'll be using grep -Ei with the regex pattern \b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b to find matches to valid email addresses (what constitutes a valid email address depends on the definition, so we'll be simplifying and considering those that match this pattern).

We will observe the following:

1. We begin by typing grep -Ei '\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b' and pressing "Enter." The absence of any filenames tells the shell that the input will be coming from stdin.
2. We pass in the input by inserting:
    * The first line of text and hitting Enter
    * The second line of text and hitting Enter
    * And so on

3. Whenever the line matches the regex, the shell will return that line and highlight the match.

![](https://s3.amazonaws.com/dq-content/392/grep_stdin.gif)

In [9]:
%%bash

sort -u >sort_stdin
a
i
u
e
o
cat sorted_stdin

# 9. Redirecting Standard Input

Some commands only take input from stdin. This is the case for tr, a command used for transliteration, i.e. a command used to replace characters with (possibly other) characters.

Typical usage of this command looks like tr old_chars new_chars, where old_chars and new_chars are strings of characters or character classes. We won't get into the details of this command, but we encourage you to explore it. Now let's see an example.

Since tr reads from stdin, we could pass in data to stdin. Instead of doing what we learned on the previous screen, we'll use a pipeline to connect the output of a command to the input of tr. We will replace:

* 0 with 1
* 2 with 3
* 4 with 5
* 6 with 7
* 8 with 9

    /home/learn$ echo "2357" | tr 02468 13579
    3357
    
We see that the only even digit in the string 2357 was replaced with its corresponding digit in 13579.

    /home/learn$ echo "1337" | tr 02468 13579
    1337
    
Since there aren't any even digits in 1337, nothing was changed.

    /home/learn$ echo "Cicada 3301" | tr C02468 c13579
    cicada 3311
    
        
As we can see just above, it works with letters too.

In order to use tr, we used echo and a pipeline; this is a very inefficient use of shell resources. And so the good people behind shell languages came up with stdin redirection. We can tell a process to read input from a file instead of the standard input.

![](https://dq-content.s3.amazonaws.com/392/Redirecting+Standard+Input.png)

The concepts are very similar to what we've seen before, only the syntax is different, as we use < for input redirection, rather than >: command 0<filename. We don't need to include 0, it is optional.

In the example below we'll be using character classes to replace all uppercase letters with their lowercase version, and we'll be using the contents of the file east as the input, instead of stdin:

    /home/learn\$ tr [:upper:] [:lower:] 0<east
    east side is the best!
    linux is the best!
    dataquest is the best!
                                                
And we can redirect the output to a file by including stdout redirection:

    /home/learn$ tr [:upper:] [:lower:] <east >small_east
    /home/learn$ cat small_east
    east side is the best!
    linux is the best!
    dataquest is the best!

# 10. Next Steps

In this lesson, we learned how redirection works under the hood.

We finished this course! We learned

* How to read and explore documentation
* How to inspect files in the filesystem
* How to use basic text processing
* About different kinds of output
* About Redirect and pipe output
* About streams and file descriptors